In [1]:
!pip install google-auth-oauthlib
!pip install oauth2client
!pip install google-auth
!pip install gspread
!pip install gspread-dataframe
!pip install -U pandasql
!pip install yfinance

In [2]:
from datetime import date
from datetime import timedelta
import pandas as pd
from pandasql import sqldf
import gspread
import gspread_dataframe as gd
from oauth2client.service_account import ServiceAccountCredentials
import yfinance as yf

In [3]:
#access google sheet file
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("stock-367302-c665cb9043f9.json", scope)
client = gspread.authorize(creds)

In [4]:
#set date variables of today and 3 years ago  
today = date.today()
strToday = today.strftime('%Y-%m-%d') #conver to string as yfinance only takes string
strBefore =  (date.today() - timedelta(days = 1095)).strftime('%Y-%m-%d')

In [5]:
#set up lists for populating each Google sheet through loop   
bankList = ["TD", "RB", "Scotia", "CIBC", "BMO", "NB"] #these are tab names in google sheet file
tickerList = ["TD.TO", "RY.TO", "BNS.TO", "CM.TO", "BMO.TO", "NA.TO"] # these are tickers of stocks

In [6]:
# populate stock data for each bank 
for bankName, ticker in zip(bankList, tickerList):
    bankData = yf.download(ticker, start=strBefore, end=strToday) #get stock information. yfinance returns pandas dataframe with Date as primary key
    bankData = sqldf('SELECT date(Date) as Date, Close, Volume from bankData', globals()) #only keep Date, Close and Volumn columns
    
    sheet = client.open("Canadian Bank Stock Price 3yrs").worksheet(bankName) #access tab in Google sheet that correspond to bank name
    rowsToAppend = bankData.values.tolist() # covert dataframe to list of rows as gspread api cannot handle dataframe
    sheet.append_rows(rowsToAppend) # Append the empty table (header is manually populated in sheets)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
